In [1]:
from nozzle.client import Client
from nozzle.util import to_hex
import os.path

# client_url = os.getenv("NOZZLE_URL", "grpc://127.0.0.1:1602")
client_url = "grpc+tcp://34.122.177.97:80"
client = Client(client_url)
print(client)


In [3]:
import time

def elapsed(start):
    return round(time.time() - start, 4)

# Processes a SQL query using the provided client and prints the timings.
# The first received batched is printed out, the rest only report the row count.
def process_query(client, query):
    print(query)

    start = time.time()
    result_stream = client.get_sql(query)
    print('time to establish stream: ', elapsed(start), 's')

    total_events = 0
    batch = next(result_stream)
    print('time for first batch ', elapsed(start), 's')
    total_events += batch.num_rows
    print(batch.to_pandas().map(to_hex))

    batch_start = time.time()
    for batch in result_stream:
        total_events += batch.num_rows
        print('received batch of ', batch.num_rows, ' rows in ', elapsed(batch_start), 's')
        batch_start = time.time()

    print('total rows: ', total_events)
    print('total time to consume the stream: ', elapsed(start), 's')
    print('rows/s: ', total_events / elapsed(start))

In [9]:
# The address of the Graph Proxy contract
l1_gns = "83A5B6C542aB20436087Fc79724bdF06ea80f836"

gns_path = "abis/l1gns.json"

In [14]:
from nozzle.util import Abi
import time

# Load a JSON ABI and get the 'PoolCreated' event
gns = Abi(gns_path)

gns_created_sig = gns.events["SubgraphPublished"].signature()

pool_created_query = f"""
    select pc.block_num,
       pc.dec['subgraphDeploymentID'] as subgraph_deployment_id,
       pc.dec['reserveRatio'] as reserve_ratio
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, '{gns_created_sig}') as dec
              from logs l
             where l.address = arrow_cast(x'{l1_gns}', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('{gns_created_sig}')) pc
"""

# Explain analyze
# print(client.get_sql("explain analyze " + query, read_all=True).to_pandas()['plan'][0])

In [13]:
process_query(client, pool_created_query)


    select pc.block_num,
       pc.dec['subgraphDeploymentID'] as subgraph_deployment_id,
       pc.dec['reserveRatio'] as reserve_ratio
      from (select l.block_num,
                   evm_decode(l.topic1, l.topic2, l.topic3, l.data, 'SubgraphPublished(uint256 indexed subgraphID,bytes32 indexed subgraphDeploymentID,uint32 reserveRatio)') as dec
              from logs l
             where l.address = arrow_cast(x'83A5B6C542aB20436087Fc79724bdF06ea80f836', 'FixedSizeBinary(20)')
               and l.topic0 = evm_topic('SubgraphPublished(uint256 indexed subgraphID,bytes32 indexed subgraphDeploymentID,uint32 reserveRatio)')) pc

time to establish stream:  0.2555 s


StopIteration: 